## Install Dependencies

In [ ]:
# by putting ! in front of the command(in ipynb files), we are executing it in the terminal
"""
For Speech to Text
"""
!pip install openai-whisper



"""
youtube-dl is a command-line program to download videos from YouTube and other video sites, and it can also be used 
as a Python module to integrate video downloading functionality into Python scripts
"""
!pip install yt-dlp

In [ ]:
import yt_dlp , os
video_url = input("Enter the URL of the YouTube video: ")

'''
%(title)s → This gets replaced with the title of the YouTube video.
%(ext)s → This gets replaced with the file extension (like .mp4, .mp3, etc.).
The . in between → Ensures the filename is formatted like video_title.mp3 (or the correct extension).
'''


downloaded_filename = None  # Global variable to store filename

def my_hook(d):
    global downloaded_filename
    if d['status'] == 'finished':  # Capture filename when download completes
        downloaded_filename = d['filename']
        print(f"Downloaded: {downloaded_filename}")

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',  # Save filename as Title.extension
    'progress_hooks': [my_hook]  # Attach hook to capture filename
}

# yt_dlp stand for 
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# Now, use the captured filename in ffmpeg command
if downloaded_filename:
    ffmpeg_cmd = f'ffmpeg -i "{downloaded_filename}" -vn -acodec libmp3lame -ab 192k -ar 44100 -y "output.wav"'
    print("Running:", ffmpeg_cmd)
    os.system(ffmpeg_cmd)

    # Clean up downloaded file
    os.remove(downloaded_filename)

# so now we finally have to do audio text , save it in file and boom we got transcript

[youtube] Extracting URL: https://youtu.be/ljMy32SxQ5w?si=tc1Yqbfx7h9uSyVH&t=18
[youtube] ljMy32SxQ5w: Downloading webpage
[youtube] ljMy32SxQ5w: Downloading tv client config
[youtube] ljMy32SxQ5w: Downloading player 91201489
[youtube] ljMy32SxQ5w: Downloading tv player API JSON
[youtube] ljMy32SxQ5w: Downloading ios player API JSON
[youtube] ljMy32SxQ5w: Downloading m3u8 information
[info] ljMy32SxQ5w: Downloading 1 format(s): 140
[download] Destination: Instinct ｜ Meaning of instinct.m4a
[download] 100% of    1.00MiB in 00:00:00 at 3.47MiB/s   Downloaded: Instinct ｜ Meaning of instinct.m4a

[FixupM4a] Correcting container of "Instinct ｜ Meaning of instinct.m4a"
Running: ffmpeg -i "Instinct ｜ Meaning of instinct.m4a" -vn -acodec libmp3lame -ab 192k -ar 44100 -y "output.wav"


### FFmpeg Command Breakdown

- `-i "{downloaded_filename}"`: Specifies the input file (the downloaded video).
- `-vn`: Removes the video stream, keeping only the audio.
- `-acodec libmp3lame`: Uses the MP3 codec for audio conversion.
- `-ab 192k`: Sets the audio bitrate to 192kbps.
- `-ar 44100`: Sets the audio sample rate to 44.1kHz.
- `-y "output.wav"`: Saves the output as "output.wav", overwriting if it already exists.

### Execution
- `os.system(ffmpeg_cmd)`: Basically executes this command in terminal.


In [ ]:
# Convert audio to text
import whisper
# Whisper provides different models with varying sizes and accuracy: 
# whisper.load_model will automatically install base(or any you specify) model
model = whisper.load_model("base")
result = model.transcribe("output.wav",fp16=False)

""" 
Basically result["segments"] is a list of segments where Whisper breaks the audio into small parts and transcribes each one separately.
It contains timestamps, text, and confidence scores for each segment
Each item in result["segments"] is a dictionary with:
Key Meaning

Each item in result["segments"] is a dictionary with:
start - Segment start time (in seconds)
end	- Segment end time (in seconds)
text - The transcribed text for this segment
no_speech_prob - Probability of no speech detected
and more ..., which you read on docs(lot of docs available on internet)
"""
# result['text'] give text that 
with open("transcript.txt", "w") as f:
    for segment in result["segments"]:
        f.write(f"[{segment['start']:.2f}s] {segment['text']}  [{segment['end']:.2f}]\n\n")


# If lot of empty lines are coming in transcribed output: 
""" 
Filter Out Low-Confidence Segments

filtered_text = ""
for segment in result["segments"]:
    if segment["no_speech_prob"] < 0.5:  # Only keep confident speech
        # that file written logic here!!
"""
# os.remove("output.wav")

#### What it achieve until: Can process short audios(especially if they are not songs) , Take longer time for long audios